# NLP: Text SPAM Classifier

# Import Dataset

In [2]:
# import libraries
import pandas as pd #handles the csv
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression # import logisticregression model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix #handles models output metric report

In [3]:
# load dataset-use pandas library
df=pd.read_csv('sms_data.csv')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [6]:
# from the head before, it can be seen there is 3 unneeded columns
# lets remove it
df=df[['v1','v2']]
df=df.rename(columns={'v1':'label','v2':'text'})
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
# so now, lets observe the class label
df['label'].value_counts()

label
ham     4825
spam     747
Name: count, dtype: int64

In [11]:
# it shows that there is only two class labels
# ham or spam
# to train a model, we need to convert the value into 
# numeric values 0 or 1
df['label']=df['label'].map({'ham':0,'spam':1})
df['label'].value_counts()

label
0    4825
1     747
Name: count, dtype: int64

In [29]:
# then in order to train model, let's split data
# into 8:2 ratio, 80% training and 20% testing
x_train,x_test, y_train, y_test=train_test_split(
    df['text'],df['label'], # y should be class label- i think?-yes, confirmed
    random_state=42,test_size=0.2, # random_state will control the data shuffling (0~42)
    stratify=df['label'] # test size is 0.2-20%, stratify- do stratified sampling- the column automatically become class label
)


In [30]:
# set up the countvectorizer and tfidfvectorizer
count_vectorizer=CountVectorizer(stop_words='english') # use prebuilt english stop words dictionary
tfidf_vectorizer=TfidfVectorizer(stop_words='english',max_df=0.7) # set condition that words can only appear in maximum 70% of documents to be included in vocabulary

Things to consider:
- CountVectorizer vs Tf-Idf Vectorizer
---------------------------------------
1-Count vectorizer only generate vectors where each  element represent the raw count of a word in the documents.

2- TfIdf vectorizer  calculates TF_IDF scores for each word, considering how often it appears in a document (term-freq) and how rare it is across all documents (idf)

3- Count good for simple count task -basic analysis

4- TFIdf considered better for text classification tasks.


In [ ]:
# Transform text data into Tf-idf-weighted document-term matrix
x_train_count=count_vectorizer.fit_transform(x_train)
x_test_count=count_vectorizer.transform(x_test)
x_train_tfidf=tfidf_vectorizer.fit_transform(x_train)
x_test_tfidf=tfidf_vectorizer.transform(x_test)

Why in transforming train and test set using different operators?

-fit_transform (let the model learn new words and transform the text into vectors)
-transform (just convert text into vectors)

so, fit transform is suitable for train set, while transform is suitable for test set

In [35]:
# train logistic regression model using CountVectorizer
logres_countv=LogisticRegression()
logres_countv.fit(x_train_count,y_train)

# test the train model using test set
y_pred_countv = logres_countv.predict(x_test_count)

# utilize accuracy score to show the model accuracy
print("CountVectorizer Accuracy: ", accuracy_score(y_test,y_pred_countv))

# show the classification report
print(classification_report(y_test,y_pred_countv))

# print the confusion matrix
print("Confusion Matrix: \n", confusion_matrix(y_test,y_pred_countv))

CountVectorizer Accuracy:  0.9766816143497757
              precision    recall  f1-score   support

           0       0.97      1.00      0.99       966
           1       1.00      0.83      0.90       149

    accuracy                           0.98      1115
   macro avg       0.99      0.91      0.95      1115
weighted avg       0.98      0.98      0.98      1115

Confusion Matrix: 
 [[966   0]
 [ 26 123]]
